In [19]:
#importing all the necessary dependencies
import os
import json
import gzip
import numpy as np
import pandas as pd
import _pickle as cPickle
import numpy as np 
from sklearn import manifold
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import Constant


In [20]:
#importing our dataset from the gz compressed file to a list
data = []
with gzip.open('meta_Appliances.json.gz') as f:
    for l in f:
        data.append(json.loads(l.strip()))
    
# total length of list, this number equals total number of products
print(len(data))

# first row of the list
print(data[0])

30459
{'category': ['Appliances', 'Parts & Accessories', 'Range Parts & Accessories'], 'description': ['', "I would like to think that anyone in business would benefit from the book because it is all about the power of connecting with and learning from people who are different than we are as a key to our success as individuals and organizations. Most of us have an aversion to strangers, but we really shouldn't because they help us to stretch our thinking, grow, and even do more remarkable things in all aspects of work and life. And the right strangers can make us more complete and filled with possibilities.", "Your friends and existing networks are tremendous assets in your growth and success, but they're simply not enough. Sometimes they are too much like you to help in solving pressing problems or creating new opportunities, and sometimes they are just too likely to confirm the things you already think or believe. And besides, relying only on friends and colleagues limits your access

In [21]:
df = pd.DataFrame.from_dict(data)
#number of product entries in our data
print(len(df))

30459


In [22]:
df.head()

,category,description,title,also_buy,brand,rank,also_view,main_cat,price,asin,tech1,image,date,feature,similar_item,details,tech2
0,"[Appliances, Parts & Accessories, Range Parts ...","[, I would like to think that anyone in busine...",The Necessity of Strangers: The Intriguing Tru...,"[0307888908, 038534936X, 1250141222, 184794080...",Alan Gregerman,"855,504inBooks(","[1250141222, 0307888908, 038534936X, 1476795924]",Books,$16.39,1118461304,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,[Breeze Collection is a wonderful book by Kim ...,Breeze,"[1906487316, 1906487014, 1906487006, 190648705...",Non Basic Stock Line,"[>#10,898 in Appliances (See top 100)]",NaN,Appliances,$32.00,1906487049,"{'Item Weight': '1.15 pounds', 'Package Dimens...",[https://images-na.ssl-images-amazon.com/image...,"April 9, 2009",NaN,NaN,NaN,NaN
2,"[Appliances, Parts & Accessories]",[This is an authorized aftermarket product. Fi...,Panasonic ADD96A1052 Blade,NaN,Panasonic,">#1,465,449 in Tools & Home Improvement (See t...",NaN,Tools & Home Improvement,NaN,6040985461,"{'Part Number': 'MSCADD96A1054', 'Item Weight'...",[https://images-na.ssl-images-amazon.com/image...,"December 26, 2014","[This is an O.E.M. Authorized part, Fits with ...",NaN,NaN,NaN
3,"[Appliances, Refrigerators, Freezers & Ice Mak...",NaN,Tupperware Freezer Square Round Container Set ...,NaN,Tupperware,"[>#39,745 in Appliances (See top 100)]",NaN,Appliances,NaN,7301113188,"{'Brand Name': 'Tupperware', 'Item Weight': '9...",NaN,"November 19, 2008",[Each 3-pc. set includes two 7/8-cup/200 mL an...,NaN,NaN,NaN
4,"[Appliances, Refrigerators, Freezers & Ice Mak...",[2 X Tupperware Pure & Fresh Unique Covered Co...,2 X Tupperware Pure & Fresh Unique Covered Coo...,NaN,Tupperware,"[>#6,118 in Appliances (See top 100)]",[B004RUGHJW],Appliances,$3.62,7861850250,"{'Brand Name': 'Tupperware', 'Model Info': 'Tu...",[https://images-na.ssl-images-amazon.com/image...,"June 5, 2016",[2 X Tupperware Pure & Fresh Unique Covered Co...,NaN,NaN,NaN


In [23]:
#We are trying to eye two Categorical variables for which we can perform multitask classification
#We are using main_cat and brandas the 2 categorical variables

In [24]:
#check Nan values in data
df.isnull().sum()


category          808
description      2306
title               0
also_buy        21829
brand             585
rank              387
also_view       19293
main_cat           97
price           10413
asin                0
tech1            4633
image           18743
date             2762
feature          4417
similar_item    13092
details         29206
tech2           30458
dtype: int64

In [25]:
#unique item in main category
df["main_cat"].unique()

array(['Books', 'Appliances', 'Tools & Home Improvement', 'Amazon Home',
       'Home Audio & Theater', 'Industrial & Scientific', 'Baby',
       'All Electronics', 'Health & Personal Care', 'Sports & Outdoors',
       'Musical Instruments', 'Automotive', 'All Beauty',
       'Arts, Crafts & Sewing', 'Pet Supplies', nan, 'Toys & Games',
       'Office Products', 'Cell Phones & Accessories', 'Computers',
       'Grocery', 'Camera & Photo'], dtype=object)

In [26]:
#count brand wise
df["brand"].value_counts()

Whirlpool                  5857
GE                         3290
Frigidaire                 2661
LG                         1262
Samsung                     978
                           ... 
Micro                         1
Lambro Industries, Inc.       1
OOK                           1
Contoure                      1
Christasm House               1
Name: brand, Length: 2764, dtype: int64

In [27]:
#A sample description
df["description"][0]

['',
 "I would like to think that anyone in business would benefit from the book because it is all about the power of connecting with and learning from people who are different than we are as a key to our success as individuals and organizations. Most of us have an aversion to strangers, but we really shouldn't because they help us to stretch our thinking, grow, and even do more remarkable things in all aspects of work and life. And the right strangers can make us more complete and filled with possibilities.",
 "Your friends and existing networks are tremendous assets in your growth and success, but they're simply not enough. Sometimes they are too much like you to help in solving pressing problems or creating new opportunities, and sometimes they are just too likely to confirm the things you already think or believe. And besides, relying only on friends and colleagues limits your access to a world of cool and powerful ideas--even if you are among the best connected people on the plane

In [28]:
#dropping all the row containing nan values in the data
df1=df.dropna(subset=['description']) 

In [29]:
df1=df1.reset_index()

In [30]:
#Description was initially an array of words, converting it to string format
for i in range(len(df1['description'])):
    df1['description'][i]=''.join(df1['description'][i])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [31]:
#Filtering out and only taking brands which occured more than 3 times
sub_df = df1[df1.groupby('brand').brand.transform('count')>3].copy()

In [32]:
#DATA CLEANING
drooping nan values from main cat and brand
sub_df=sub_df.dropna(subset=['main_cat','brand'])


In [33]:
# convert to categorical data for training
labels_brand=to_categorical(sub_df["brand"].astype("category").cat.codes)
labels_main_cat=to_categorical(sub_df["main_cat"].astype("category").cat.codes)

In [34]:
#labels_brand is the ONEHOT ENCODING of the brands.
#There are 25327 PRODUCT ENTRIES and 503 DISTINCT BRANDS
labels_brand.shape

(25327, 503)

In [35]:
#labels_main_cat is the ONEHOT ENCODING of the brands.
#There are 25327 PRODUCT ENTRIES and 20 DISTINCT BRANDS
labels_main_cat.shape

(25327, 20)

In [36]:
# list of all the description
docs=sub_df["description"].tolist()

In [37]:
#maximum length in list
len(max(docs, key = len))

85069

In [38]:
#We will limit of pad all the description to 2000 words
MAX_SEQUENCE_LENGTH = 2000 

In [40]:
#other parameters
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 50
VALIDATION_SPLIT = 0.2

In [42]:
#getting embeddings from glove
def unpickle(filename):
    f = open(filename, "rb")
    d = cPickle.load(f)
    f.close()
    return d

glove_data=unpickle("glove_new (1).txt")
embeddings_index=glove_data["embeddings_index"]

In [45]:
type(embeddings_index["the"])

numpy.ndarray

In [46]:
#embeddings_index has 50d embeddings for 400000 words in numpy array format
len(embeddings_index)

400000

In [48]:
#TOKENIZING words to numbers
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(docs)
sequences = tokenizer.texts_to_sequences(docs)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', data.shape)
print('Shape of brand label tensor:', labels_brand.shape)
print('Shape of main category label tensor:', labels_main_cat.shape)

Found 72598 unique tokens.
Shape of data tensor: (25327, 2000)
Shape of brand label tensor: (25327, 503)
Shape of main category label tensor: (25327, 20)


In [49]:
#SAVING THE DATASET FOR FUTURE USE
sub_df.to_csv("final_df.csv")

In [50]:
# split the data into a training set and a validation set
indices = np.arange(data.shape[0]) 
np.random.shuffle(indices) 
data = data[indices]
labels_brand = labels_brand[indices]
labels_main_cat = labels_main_cat[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train_brand = labels_brand[:-num_validation_samples]
y_train_main_cat = labels_main_cat[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val_brand = labels_brand[-num_validation_samples:]
y_val_main_cat = labels_main_cat[-num_validation_samples:]

In [51]:
# prepare embedding matrix from the Tokenized Description data
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [52]:
#PREPERING THE MODEL
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)
print('Training model.')

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(256, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(512, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(512, activation='relu')(x)
#two branches
brand = Dense(len(labels_brand[0]), activation='softmax')(x)
main_category = Dense(len(labels_main_cat[0]), activation='softmax')(x)

#model = Model(sequence_input, preds)
model = Model(inputs=sequence_input, outputs=[brand, main_category])



Training model.


In [53]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 2000)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 2000, 50)     1000000     input_1[0][0]                    
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 1996, 128)    32128       embedding[0][0]                  
__________________________________________________________________________________________________
max_pooling1d (MaxPooling1D)    (None, 399, 128)     0           conv1d[0][0]                     
______________________________________________________________________________________________

In [54]:
model.compile(optimizer='rmsprop',
              loss={'dense_2': 'categorical_crossentropy', 'dense_3': 'categorical_crossentropy'},
              loss_weights={'dense_2': 1., 'dense_3': 1.},
              metrics={'dense_2': 'acc', 'dense_3': 'acc'})

In [ ]:
# train your model
model.fit(x_train,
          {'dense_2': y_train_brand, 'dense_3': y_train_main_cat},
          epochs=30, batch_size=64,validation_data=(x_val,{'dense_2': y_val_brand, 'dense_3': y_val_main_cat}))

In [ ]:
#get the weights of first layers 
#below comparision will show how the GloVe embeddings have finetuned for our Description corpus
weights = model.layers[1].get_weights()[0]

In [ ]:
weights[1]

In [ ]:
embedding_matrix[1]

In [ ]:
weights.shape

In [ ]:
#SAVING THE MODEL TO JSON FILE
from keras.models import model_from_json
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
#RECALLING MODEL FROM JSON FORMAT
from tensorflow.keras.models import model_from_json
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

In [ ]:
#WE EXTRACT EMBEDDINGS FROM THE LAST COMMON LAYER OF OUR MULTITASK CLASSIFICATION NET
from keras.models import Model

model = loaded_model  # include here your original model
#edit layer_name referring to model.summary() in case of errors 
layer_name = 'dense_1'
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict(data)

In [ ]:
intermediate_output.shape

In [ ]:
#SAVING ABOVE OBTAINED EMBEDDINGS IN A FILE FOR FUTURE USE
import pickle

with open("embeddings.pkl","wb") as f:
    pickle.dump(intermediate_output,f)